In [1]:
import pandas as pd
import plotly.express as px

In [4]:
df_kronyx = pd.read_csv("../data/Kronyx67.csv")
df_gotwiller = pd.read_csv("../data/Gautier.csv")

# 2. Aperçu rapide
df_kronyx.head()      

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1759297343,"01 Oct 2025, 05:42",HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821
1,1759297204,"01 Oct 2025, 05:40",Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN
2,1759297023,"01 Oct 2025, 05:37",Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a
3,1759296861,"01 Oct 2025, 05:34",Grace Power,NaN,Siren Song,NaN,Siren Song,NaN
4,1759263010,"30 Sep 2025, 20:10",Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f


In [5]:
df_gotwiller.head()  

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1756792590,"02 Sep 2025, 05:56",eyeto8,NaN,Boy Meets the World,NaN,Sound of Time Passing,NaN
1,1756792013,"02 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,The Shift,NaN
2,1756791598,"02 Sep 2025, 05:39",eyeto8,NaN,The Old Electronic Eldorado,NaN,Audentity,NaN
3,1756791295,"02 Sep 2025, 05:34",eyeto8,NaN,The Old Electronic Eldorado,NaN,Urgent,NaN
4,1756705616,"01 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,Lost Disco Thrills,NaN


In [11]:
for df in [df_kronyx, df_gotwiller]:
    df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")
    # Ajouter des colonnes utiles
    df["date"] = df["utc_time"].dt.date
    df["hour"] = df["utc_time"].dt.hour
    df["weekday"] = df["utc_time"].dt.day_name()
    df["minute"] = df["utc_time"].dt.minute

In [10]:
df_kronyx.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,hour,weekday,minute
0,1759297343,2025-10-01 05:42:00,HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821,2025-10-01,5,Wednesday,42
1,1759297204,2025-10-01 05:40:00,Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN,2025-10-01,5,Wednesday,40
2,1759297023,2025-10-01 05:37:00,Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a,2025-10-01,5,Wednesday,37
3,1759296861,2025-10-01 05:34:00,Grace Power,NaN,Siren Song,NaN,Siren Song,NaN,2025-10-01,5,Wednesday,34
4,1759263010,2025-09-30 20:10:00,Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f,2025-09-30,20,Tuesday,10


## Visualisations